# This notebook demostrates the effect of input catalog ordering in NWayMatch and MultiMatch

It assumes that you:

    1. Have downloaded the data here:  https://lsst.ncsa.illinois.edu/~yusra/nway-matcher/
    2. Use the script pq2afw.py to make afw source catalogs for MultiMatch


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import glob
import os
import numpy as np
from nway import NWayMatch

### Set up the inputs

In [ ]:
DATADIR = "."
SOURCE_TABLEFILES = glob.glob(os.path.join(DATADIR, "sourceTable-00*.parq"))
SOURCE_CATFILES = glob.glob(os.path.join(DATADIR, "sourceTable-*.fits"))
VISIT_IDS = np.arange(len(SOURCE_TABLEFILES))

REF_DIR = (150., 2.)   # RA, DEC in deg
REGION_SIZE = (3., 3.) # in Deg
CELL_SIZE = 1. / (3600*2) # in Deg = 0.5"
SUBREGION_SIZE = 1350 # in Pixels
PIXEL_R2CUT = 1.

#### Create two NWayMatch matchers

Give the the same data, but reverse the order of the input files in one case

In [ ]:
nWay = NWayMatch.create(REF_DIR, REGION_SIZE, CELL_SIZE, pixelR2Cut=PIXEL_R2CUT, subRegionSize=SUBREGION_SIZE)
nWay2 = NWayMatch.create(REF_DIR, REGION_SIZE, CELL_SIZE, pixelR2Cut=PIXEL_R2CUT, subRegionSize=SUBREGION_SIZE)

In [ ]:
nWay.reduceData(SOURCE_TABLEFILES, VISIT_IDS)
nWay2.reduceData(SOURCE_TABLEFILES[::-1], VISIT_IDS[::-1])

In [ ]:
oDict = nWay.analyzeSubregion(10, 10, True)
oDict2 = nWay2.analyzeSubregion(10, 10, True)

### Show that they have the same number of clusters and objects

In [ ]:
print(oDict['srd'].nClusters, oDict2['srd'].nClusters) 
print(oDict['srd'].nObjects, oDict2['srd'].nObjects)

### Set up a function to run MultiMatch

In [ ]:
import lsst.afw.table as afwTable
import lsst.geom as geom
from astropy import table
from astropy.table import Table

def runMultiMatch(inputCats, outFile):
    match_radius = geom.Angle(0.5, geom.arcseconds)
    
    schema = afwTable.SourceTable.makeMinimalSchema()
    mmatch = afwTable.MultiMatch(schema, dataIdFormat={'iCat': np.int32},
                                 radius=match_radius,
                                 RecordClass=afwTable.SimpleRecord)

    for i, catFile in enumerate(inputCats):
        print("Adding %s" % catFile)
        cat = afwTable.SourceCatalog.readFits(catFile)
        mmatch.add(catalog=cat, dataId=dict(iCat=i))

    matchCat = mmatch.finish()
    matchCat.writeFits(outFile)

#### Run it twice, once with the catalogs in reverse order

In [ ]:
mm1 = runMultiMatch(SOURCE_CATFILES[0:5], 'mm_orig.fits')
tt1 = Table.read('mm_orig.fits')

In [ ]:
mm2 = runMultiMatch(SOURCE_CATFILES[0:5][::-1], 'mm_revr.fits')
tt2 = Table.read('mm_revr.fits')

#### Show that the number of associations changes

In [ ]:
print(len(tt1), len(tt2))